### Import Libraries

In [35]:
import os
import torch
import numpy as np
import lpips
import cv2

### Import Metrics

In [30]:
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

#### Gaussian Blur Deconvolution

In [37]:
# === Set your directories ===
ground_truth_dir = r"C:\Users\sneha\OneDrive\Documents\test high quality satellite images (.tif)"
deconvolved_dir = r"C:\Users\sneha\OneDrive\Documents\gaussian rl deconvolved test images"

# === Get sorted image filenames ===
gt_images = sorted(os.listdir(ground_truth_dir))
dc_images = sorted(os.listdir(deconvolved_dir))

# === Initialize accumulators ===
total_psnr = 0.0
total_ssim = 0.0
total_lpips = 0.0
num_images = len(gt_images)

# === Initialize LPIPS model ===
lpips_model = lpips.LPIPS(net='vgg').to('cuda' if torch.cuda.is_available() else 'cpu')
lpips_model.eval()

# === Loop through images ===
for gt_name, dc_name in zip(gt_images, dc_images):
    gt_path = os.path.join(ground_truth_dir, gt_name)
    dc_path = os.path.join(deconvolved_dir, dc_name)

    # Read and convert to RGB
    gt_img = cv2.cvtColor(cv2.imread(gt_path), cv2.COLOR_BGR2RGB)
    dc_img = cv2.cvtColor(cv2.imread(dc_path), cv2.COLOR_BGR2RGB)

    # Resize if needed
    if gt_img.shape != dc_img.shape:
        dc_img = cv2.resize(dc_img, (gt_img.shape[1], gt_img.shape[0]))

    # === PSNR and SSIM ===
    total_psnr += psnr(gt_img, dc_img, data_range=255)
    total_ssim += ssim(gt_img, dc_img, data_range=255, channel_axis=-1)

    # === LPIPS ===
    gt_norm = gt_img.astype(np.float32) / 127.5 - 1.0  # Normalize to [-1, 1]
    dc_norm = dc_img.astype(np.float32) / 127.5 - 1.0

    gt_tensor = torch.tensor(gt_norm).permute(2, 0, 1).unsqueeze(0).float()
    dc_tensor = torch.tensor(dc_norm).permute(2, 0, 1).unsqueeze(0).float()

    if torch.cuda.is_available():
        gt_tensor = gt_tensor.cuda()
        dc_tensor = dc_tensor.cuda()

    with torch.no_grad():
        lpips_score = lpips_model(gt_tensor, dc_tensor).item()
    total_lpips += lpips_score

# === Final averages ===
avg_psnr = total_psnr / num_images
avg_ssim = total_ssim / num_images
avg_lpips = total_lpips / num_images

print(f"\n=== Evaluation over {num_images} images ===")
print(f"Average PSNR:  {avg_psnr:.2f} dB")
print(f"Average SSIM:  {avg_ssim:.4f}")
print(f"Average LPIPS: {avg_lpips:.4f}")

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: C:\Users\sneha\anaconda\envs\gpu_env\lib\site-packages\lpips\weights\v0.1\vgg.pth

=== Evaluation over 77 images ===
Average PSNR:  10.88 dB
Average SSIM:  0.2171
Average LPIPS: 0.8315


#### Paraxial Lens Deconvolution

In [39]:
# === Set your directories ===
ground_truth_dir = r"C:\Users\sneha\OneDrive\Documents\test high quality satellite images (.tif)"
deconvolved_dir = r"C:\Users\sneha\OneDrive\Documents\paraxial lens rl deconvolved test images"

# === Get sorted image filenames ===
gt_images = sorted(os.listdir(ground_truth_dir))
dc_images = sorted(os.listdir(deconvolved_dir))

# === Initialize accumulators ===
total_psnr = 0.0
total_ssim = 0.0
total_lpips = 0.0
num_images = len(gt_images)

# === Initialize LPIPS model ===
lpips_model = lpips.LPIPS(net='vgg').to('cuda' if torch.cuda.is_available() else 'cpu')
lpips_model.eval()

# === Loop through images ===
for gt_name, dc_name in zip(gt_images, dc_images):
    gt_path = os.path.join(ground_truth_dir, gt_name)
    dc_path = os.path.join(deconvolved_dir, dc_name)

    # Read and convert to RGB
    gt_img = cv2.cvtColor(cv2.imread(gt_path), cv2.COLOR_BGR2RGB)
    dc_img = cv2.cvtColor(cv2.imread(dc_path), cv2.COLOR_BGR2RGB)

    # Resize if needed
    if gt_img.shape != dc_img.shape:
        dc_img = cv2.resize(dc_img, (gt_img.shape[1], gt_img.shape[0]))

    # === PSNR and SSIM ===
    total_psnr += psnr(gt_img, dc_img, data_range=255)
    total_ssim += ssim(gt_img, dc_img, data_range=255, channel_axis=-1)

    # === LPIPS ===
    gt_norm = gt_img.astype(np.float32) / 127.5 - 1.0  # Normalize to [-1, 1]
    dc_norm = dc_img.astype(np.float32) / 127.5 - 1.0

    gt_tensor = torch.tensor(gt_norm).permute(2, 0, 1).unsqueeze(0).float()
    dc_tensor = torch.tensor(dc_norm).permute(2, 0, 1).unsqueeze(0).float()

    if torch.cuda.is_available():
        gt_tensor = gt_tensor.cuda()
        dc_tensor = dc_tensor.cuda()

    with torch.no_grad():
        lpips_score = lpips_model(gt_tensor, dc_tensor).item()
    total_lpips += lpips_score

# === Final averages ===
avg_psnr = total_psnr / num_images
avg_ssim = total_ssim / num_images
avg_lpips = total_lpips / num_images

print(f"\n=== Evaluation over {num_images} images ===")
print(f"Average PSNR:  {avg_psnr:.2f} dB")
print(f"Average SSIM:  {avg_ssim:.4f}")
print(f"Average LPIPS: {avg_lpips:.4f}")

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: C:\Users\sneha\anaconda\envs\gpu_env\lib\site-packages\lpips\weights\v0.1\vgg.pth

=== Evaluation over 77 images ===
Average PSNR:  3.79 dB
Average SSIM:  0.0844
Average LPIPS: 0.2802


#### Plano Convex Deconvolution

In [41]:
# === Set your directories ===
ground_truth_dir = r"C:\Users\sneha\OneDrive\Documents\test high quality satellite images (.tif)"
deconvolved_dir = r"C:\Users\sneha\OneDrive\Documents\plano convex lens rl deconvolved test images"

# === Get sorted image filenames ===
gt_images = sorted(os.listdir(ground_truth_dir))
dc_images = sorted(os.listdir(deconvolved_dir))

# === Initialize accumulators ===
total_psnr = 0.0
total_ssim = 0.0
total_lpips = 0.0
num_images = len(gt_images)

# === Initialize LPIPS model ===
lpips_model = lpips.LPIPS(net='vgg').to('cuda' if torch.cuda.is_available() else 'cpu')
lpips_model.eval()

# === Loop through images ===
for gt_name, dc_name in zip(gt_images, dc_images):
    gt_path = os.path.join(ground_truth_dir, gt_name)
    dc_path = os.path.join(deconvolved_dir, dc_name)

    # Read and convert to RGB
    gt_img = cv2.cvtColor(cv2.imread(gt_path), cv2.COLOR_BGR2RGB)
    dc_img = cv2.cvtColor(cv2.imread(dc_path), cv2.COLOR_BGR2RGB)

    # Resize if needed
    if gt_img.shape != dc_img.shape:
        dc_img = cv2.resize(dc_img, (gt_img.shape[1], gt_img.shape[0]))

    # === PSNR and SSIM ===
    total_psnr += psnr(gt_img, dc_img, data_range=255)
    total_ssim += ssim(gt_img, dc_img, data_range=255, channel_axis=-1)

    # === LPIPS ===
    gt_norm = gt_img.astype(np.float32) / 127.5 - 1.0  # Normalize to [-1, 1]
    dc_norm = dc_img.astype(np.float32) / 127.5 - 1.0

    gt_tensor = torch.tensor(gt_norm).permute(2, 0, 1).unsqueeze(0).float()
    dc_tensor = torch.tensor(dc_norm).permute(2, 0, 1).unsqueeze(0).float()

    if torch.cuda.is_available():
        gt_tensor = gt_tensor.cuda()
        dc_tensor = dc_tensor.cuda()

    with torch.no_grad():
        lpips_score = lpips_model(gt_tensor, dc_tensor).item()
    total_lpips += lpips_score

# === Final averages ===
avg_psnr = total_psnr / num_images
avg_ssim = total_ssim / num_images
avg_lpips = total_lpips / num_images

print(f"\n=== Evaluation over {num_images} images ===")
print(f"Average PSNR:  {avg_psnr:.2f} dB")
print(f"Average SSIM:  {avg_ssim:.4f}")
print(f"Average LPIPS: {avg_lpips:.4f}")

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: C:\Users\sneha\anaconda\envs\gpu_env\lib\site-packages\lpips\weights\v0.1\vgg.pth

=== Evaluation over 77 images ===
Average PSNR:  4.00 dB
Average SSIM:  0.0836
Average LPIPS: 0.2958
